In [25]:
import pandas as pd
import sqlite3
import os
import requests
from bs4 import BeautifulSoup

In [4]:
# connect to db
con = sqlite3.connect('rijksstudio.db')
cur = con.cursor()

In [3]:
# list of all paintings
paintings = os.listdir("static/images/paintings/")

['SK-A-2152.png',
 'SK-A-2344.png',
 'SK-A-385.png',
 'SK-A-390.png',
 'SK-A-4052.png',
 'SK-A-1857.png',
 'SK-C-134.png',
 'SK-A-4867.png',
 'SK-A-4118.png',
 'SK-A-3930.png',
 'SK-A-3924.png',
 'SK-A-3059.png',
 'SK-A-2150.png',
 'SK-A-4290.png',
 'SK-A-4054.png',
 'SK-A-180.png',
 'SK-A-4646.png',
 'SK-A-3103.png',
 'SK-A-2962.png',
 'SK-A-3467.png',
 'SK-C-291.png',
 'SK-A-2180.png',
 'SK-A-1649.png',
 'SK-C-535.png',
 'SK-A-752.png',
 'SK-A-4839.png',
 'SK-A-4152.png',
 'SK-C-187.png',
 'SK-A-690.png',
 'SK-A-3995.png',
 'SK-A-4157.png',
 'SK-C-147.png',
 'SK-A-718.png',
 'SK-A-98.png',
 'SK-A-335.png',
 'SK-A-137.png',
 'SK-C-152.png',
 'SK-C-185.png',
 'SK-A-4830.png',
 'SK-C-177.png',
 'SK-C-229.png',
 'SK-A-1627.png',
 'SK-A-113.png',
 'SK-A-3347.png',
 'SK-C-610.png',
 'SK-A-662.png',
 'SK-A-1751.png',
 'SK-C-149.png',
 'SK-A-4163.png',
 'SK-C-1458.png',
 'SK-C-1672.png',
 'SK-A-86.png',
 'SK-A-4981.png',
 'SK-A-3988.png',
 'SK-A-3962.png',
 'SK-A-93.png',
 'SK-C-206.png',
 '

In [78]:
# Returns df with average coordinates and confidence of 2 objects
def average_coords(GV_obj, HF_obj):
    row = pd.DataFrame()
    row['painting_id'] = [GV_obj['painting_id']]
    row['object_name'] = ['']
    row['confidence'] = [(GV_obj['confidence']+HF_obj['confidence'])/2]
    row['min_x'] = [(GV_obj['min_x']+HF_obj['min_x'])/2]
    row['min_y'] = [(GV_obj['min_y']+HF_obj['min_y'])/2]
    row['max_x'] = [(GV_obj['max_x']+HF_obj['max_x'])/2]
    row['max_y'] = [(GV_obj['max_y']+HF_obj['max_y'])/2]
    
    return row

In [162]:
# Updates db by 'removing' or adding objects
def update_db(row, cur, decision):
    
    if decision in ['both', 'HF']:
        # 'delete' object from db: keep Google Vision object for backtracking but change source name
        cur.execute(f"""UPDATE Objects 
                        SET source = 'GV_LOG' 
                        WHERE painting_id = '{row['painting_id']}' 
                        AND object_name = '{row['object_name']}' 
                        AND source = 'GOOGLE_VISION'
                        """)
        
    if decision in ['both', 'GV']:
        # 'delete' object from db: keep DETR Hugging Face object for backtracking but change source name
        cur.execute(f"""UPDATE Objects 
                        SET source = 'HF_LOG' 
                        WHERE painting_id = '{row['painting_id']}' 
                        AND object_name = '{row['object_name']}' 
                        AND source = 'HUGGING_FACE'
                        """)
        
    # add new object value to table if both objects are averaged
    if decision == 'both':
        # add new averaged object
        cur.execute(f"""INSERT INTO Objects
                        VALUES ("{row['painting_id']}", 
                                "{row['object_name']}", 
                                "{row['confidence']}", 
                                {row['min_x']}, 
                                {row['min_y']},
                                {row['max_x']},
                                {row['max_y']},
                                {row['source']},
                                {row['total_clicks']},
                                '{row['flags']}')""")

In [160]:
# Determine and handle if 2 objects found by different models are the same
def compare_objects(painting): 
    # get all CV objects for painting
    q = cur.execute(f"""SELECT * FROM Objects WHERE painting_id = '{painting}' AND source = 'GOOGLE_VISION' OR source = 'HUGGING_FACE'""")
    cols = [column[0] for column in q.description]
    df= pd.DataFrame.from_records(data = q.fetchall(), columns = cols)
    
    # split up df by vision model
    GV = df[df['source']=='GOOGLE_VISION']
    HF = df[df['source']=='HUGGING_FACE']

    # check if objects are the same as DETR huggingface
    for index, GV_obj in GV.iterrows():
        for index, HF_obj in HF.iterrows():
            # check if two objects are in the same 10% range
            in_range = [HF_obj['min_x']*0.9 <= GV_obj['min_x'] <= HF_obj['min_x']*1.1, HF_obj['min_y']*0.9 <= GV_obj['min_y'] <= HF_obj['min_y']*1.1, HF_obj['max_x']*0.9 <= GV_obj['max_x'] <= HF_obj['max_x']*1.1, HF_obj['max_y']*0.9 <= GV_obj['max_y'] <= HF_obj['max_y']*1.1]

            # consider same object if 3/4 coordinates are within 10% range of each other
            if sum(in_range) >= 3:
                # check if objects have the same name
                if (GV_obj['object_name'] ==  HF_obj['object_name']):
                    # get average coordinates and score
                    row = average_coords(GV_obj, HF_obj)

                    # update object values
                    row['object_name'], row['source'], row['total_clicks'], row['flags'] = GV_obj['object_name'], 'GVHF', 0, 0
                    update_db(row, cur, 'both')
    
                # if not the same name determine which object to keep based on search hits
                else:
                    # get number google search results for objects
                    GV_results = get_google_hits(GV_obj['object_name'])
                    HF_results = get_google_hits(HF_obj['object_name'])
                    
                    # assumption: less google hits is more specific object so that object kept    
                    if HF_results>GV_results:
                        # keep google vision object
                        update_db(GV_obj, cur, 'GV')
                    else:
                        # keep hugging face object if less search results or no response
                        update_db(HF_obj, cur, 'HF')


In [161]:
# Return number of Google Search results for object
def get_google_hits(obj):
    headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'}
    URL     = "https://www.google.com/search?q=" + obj
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

    result = requests.get(URL, headers=headers,cookies=cookies)  
    soup = BeautifulSoup(result.text, 'html.parser')
    
    try:
        results_text = soup.find("div", {"id": "result-stats"}).find(text=True, recursive=False)
        nr_results = ''.join([num for num in total_results_text if num.isdigit()])  # get number of results from text
    except:
        return False
    
    return nr_results


In [100]:
for painting in paintings:
    compare_objects(painting)

gv:  ['vase' 'fruit']
hf:  ['vase']
same:   vase vase
false:   fruit vase
102.94306378811598 1073.101319551468 422.21292823553085 1339.280341386795
2278.490966796875 944.0518798828125 2586.72021484375 1312.1650390625
gv:  ['person' 'box' 'food' 'food']
hf:  ['person']
same:   person person
false:   box person
1754.2666543126106 2107.7465941905975 2078.2564920783043 2373.311691045761
643.003173828125 390.1028747558594 1694.60546875 2518.25732421875
false:   food person
130.32211382314563 1518.50066614151 822.0274681150913 1987.7163352966309
643.003173828125 390.1028747558594 1694.60546875 2518.25732421875
false:   food person
595.2822888791561 1254.118884921074 1110.7153255045414 1830.4292891025543
643.003173828125 390.1028747558594 1694.60546875 2518.25732421875
gv:  ['person' 'person' 'person' 'person' 'person' 'person' 'person' 'person'
 'person' 'furniture']
hf:  ['skateboard' 'person' 'person' 'person' 'tie' 'person' 'handbag' 'person'
 'person' 'person']
false:   person skateboard

false:   person person
1232.12709659338 1492.4456179141998 1498.1061773896217 2132.4263513088226
1009.7167358398438 1190.999755859375 1299.2171630859375 1994.152099609375
false:   person person
1232.12709659338 1492.4456179141998 1498.1061773896217 2132.4263513088226
749.1002197265625 1641.009765625 1117.02978515625 2231.096923828125
same:   person person
false:   person person
741.1987138688564 1631.820648908615 1113.5811213850975 2188.3119642734528
435.22991943359375 1367.5091552734375 509.5318603515625 1522.8192138671875
false:   person person
741.1987138688564 1631.820648908615 1113.5811213850975 2188.3119642734528
1487.18212890625 1638.89306640625 1718.53125 2123.9619140625
false:   person person
741.1987138688564 1631.820648908615 1113.5811213850975 2188.3119642734528
1671.0716552734375 1297.0078125 1932.40478515625 1624.458740234375
false:   person person
741.1987138688564 1631.820648908615 1113.5811213850975 2188.3119642734528
1585.96923828125 1461.775634765625 1744.05517578125

2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
1132.6219482421875 1413.5736083984375 1274.993896484375 1755.8919677734375
false:   person person
2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
159.20188903808594 988.5234375 264.2117004394531 1121.90966796875
false:   person person
2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
1751.057373046875 1498.0245361328125 1871.1961669921875 1804.3199462890625
same2:   person horse
same:   person person
false:   person person
2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
1025.1104736328125 1511.974853515625 1184.5885009765625 1764.71435546875
false:   person person
2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
1244.148681640625 1333.3955078125 1370.031494140625 1563.759765625
false:   person person
2187.437564134598 1468.5212416648865 2402.8678238391876 1818.1223149299622
810.2444458007812 1576.914672851562

false:   person person
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
1244.148681640625 1333.3955078125 1370.031494140625 1563.759765625
false:   person person
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
810.2444458007812 1576.9146728515625 918.0368041992188 1820.807861328125
false:   person horse
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
851.653076171875 1346.5709228515625 1030.8326416015625 1571.5828857421875
false:   person person
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
647.8836669921875 1552.2039794921875 735.7329711914062 1769.495361328125
false:   person person
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
1295.5582275390625 1441.6181640625 1431.08935546875 1707.131591796875
false:   person person
1014.8853063583374 1518.5789192914963 1191.2629008293152 1763.2251801490784
1428.8548583984375 1461.820556640625 1557.59680175

743.1738823652267 939.0848137140274 1064.77789580822 1320.2088059186935
1661.6529541015625 953.5586547851562 1752.724609375 1178.4765625
false:   animal airplane
1008.344367146492 923.1234284639359 1223.202645778656 1272.2779369354248
935.4055786132812 554.6651000976562 961.670654296875 563.9073486328125
false:   animal airplane
1008.344367146492 923.1234284639359 1223.202645778656 1272.2779369354248
349.9970703125 572.4119873046875 389.3514099121094 584.7166748046875
false:   animal cow
1008.344367146492 923.1234284639359 1223.202645778656 1272.2779369354248
1802.903076171875 1056.3082275390625 2076.066162109375 1207.7650146484375
false:   animal person
1008.344367146492 923.1234284639359 1223.202645778656 1272.2779369354248
1325.5086669921875 1025.1082763671875 1367.230712890625 1127.5845947265625
false:   animal person
1008.344367146492 923.1234284639359 1223.202645778656 1272.2779369354248
893.2976684570312 907.3325805664062 1022.7520751953125 1191.1859130859375
false:   animal air

...

false:   wheel horse
1538.5903418064117 809.6756941080093 1714.1823470592499 970.5489277839661
478.9311828613281 709.6605834960938 666.1771240234375 963.873291015625
false:   wheel horse
1538.5903418064117 809.6756941080093 1714.1823470592499 970.5489277839661
606.03173828125 818.1085205078125 796.1400146484375 997.2969360351562
false:   wheel horse
1538.5903418064117 809.6756941080093 1714.1823470592499 970.5489277839661
317.60321044921875 702.0455932617188 576.439697265625 956.8997192382812
false:   wheel horse
1538.5903418064117 809.6756941080093 1714.1823470592499 970.5489277839661
1151.8909912109375 698.678955078125 1406.5330810546875 980.3848266601562
false:   wheel horse
1538.5903418064117 809.6756941080093 1714.1823470592499 970.5489277839661
1176.230224609375 696.981689453125 1442.0048828125 994.8960571289062
gv:  ['person' 'person' 'person' 'person' 'person' 'person' 'person'
 'tableware' 'racket']
hf:  ['cup' 'person' 'person' 'cup' 'person' 'person' 'cup' 'person' 'person'


434.6751708984375 774.6183471679688 599.0828247070312 876.1322021484375
false:   person bird
680.3344376385212 1591.1276321411133 878.4552477300167 2093.9501953125
828.73779296875 227.57662963867188 1279.8878173828125 986.1875610351562
false:   person person
680.3344376385212 1591.1276321411133 878.4552477300167 2093.9501953125
1519.8209228515625 1333.634765625 1558.46728515625 1456.512451171875
false:   person dog
680.3344376385212 1591.1276321411133 878.4552477300167 2093.9501953125
1112.927490234375 1852.74365234375 1414.5821533203125 2089.13427734375
false:   person bird
680.3344376385212 1591.1276321411133 878.4552477300167 2093.9501953125
1000.2003173828125 1656.011962890625 1096.2591552734375 1839.189697265625
false:   bird chair
1450.001634657383 1951.8898544311523 1867.4571812152863 2368.6305541992188
322.04840087890625 1777.229248046875 770.0557250976562 2650.305908203125
same:   bird bird
false:   bird person
1450.001634657383 1951.8898544311523 1867.4571812152863 2368.63055

same2:   animal person
false:   person dog
889.4473287463188 300.4705462753773 1914.3072777986526 1683.0152971744537
1222.7689208984375 2061.558349609375 1575.9052734375 2636.735595703125
same:   person person
false:   person person
889.4473287463188 300.4705462753773 1914.3072777986526 1683.0152971744537
1.7832008600234985 809.1965942382812 1420.9931640625 2687.401123046875
gv:  []
hf:  ['cat' 'cat']
gv:  ['chair' 'person' 'person' 'hat' 'person' 'person' 'couch' 'clothing'
 'picture frame' 'person']
hf:  ['chair' 'person' 'person' 'person' 'person']
same:   chair chair
false:   chair person
19.107711410149932 1486.9584143161774 570.999621450901 2396.6705799102783
1482.70703125 818.9290161132812 1921.9713134765625 2112.67431640625
false:   chair person
19.107711410149932 1486.9584143161774 570.999621450901 2396.6705799102783
633.2570190429688 953.5013427734375 951.184326171875 1355.5523681640625
false:   chair person
19.107711410149932 1486.9584143161774 570.999621450901 2396.67057991

2271.7738151550293 1110.7590024471283 2489.449232816696 1635.8107643127441
1345.925048828125 1184.96875 1664.8924560546875 1351.3323974609375
same2:   furniture chair
false:   furniture person
2271.7738151550293 1110.7590024471283 2489.449232816696 1635.8107643127441
576.2225341796875 875.91943359375 1101.2958984375 1820.0299072265625
false:   furniture person
2271.7738151550293 1110.7590024471283 2489.449232816696 1635.8107643127441
1371.903076171875 872.009765625 2103.642822265625 1989.698974609375
gv:  ['person' 'toy']
hf:  ['person']
same:   person person
same2:   toy person
gv:  ['hat' 'person' 'picture frame']
hf:  ['person']
false:   hat person
444.85002364218235 241.6802942752838 1293.0411887168884 533.9856073260307
3.218442440032959 250.65370178222656 1883.04833984375 2456.08447265625
same:   person person
false:   picture frame person
1497.6777822375298 456.20977878570557 1852.898456275463 974.067747592926
3.218442440032959 250.65370178222656 1883.04833984375 2456.08447265625

392.5850524902344 843.3301391601562 724.5322265625 1828.0665283203125
false:   hat dog
1893.361508846283 826.3436190187931 2194.754183292389 1022.8543576896191
1411.542724609375 1611.3839111328125 1837.24609375 1906.990234375
false:   hat person
1893.361508846283 826.3436190187931 2194.754183292389 1022.8543576896191
1632.3065185546875 868.7911987304688 2413.16845703125 2092.3134765625
false:   person person
1265.4927670955658 987.3980939388275 1689.8839473724365 1307.5291333794594
2311.279296875 625.0042724609375 2869.549560546875 2013.8507080078125
false:   person person
1265.4927670955658 987.3980939388275 1689.8839473724365 1307.5291333794594
851.0784912109375 1124.4107666015625 1124.53955078125 1888.9241943359375
false:   person person
1265.4927670955658 987.3980939388275 1689.8839473724365 1307.5291333794594
184.2194061279297 947.3118896484375 449.16644287109375 1791.6248779296875
false:   person person
1265.4927670955658 987.3980939388275 1689.8839473724365 1307.5291333794594
39

In [163]:
objects = pd.DataFrame(json.load(open("objects_vision.json")))
objects = objects.append(pd.DataFrame(json.load(open("objects_hugging.json"))))
objects

,painting_id,object_name,confidence,min_x,min_y,max_x,max_y
0,RP-F-2004-71.png,tableware,0.714782,1549.506351,1640.297090,2465.202644,1997.612133
1,RP-F-2004-71.png,pitcher,0.686795,1128.606110,1069.994700,1688.415783,1872.972900
2,RP-F-2004-71.png,tableware,0.576082,950.463439,1380.600437,1274.653524,1831.608840
3,RP-F-2004-71.png,tableware,0.542288,126.578440,1295.619819,751.920914,1919.675842
4,RP-F-2014-7-1-3.png,vase,0.731736,635.351464,479.213008,1092.309430,1159.795865
...,...,...,...,...,...,...,...
477,SK-C-301.png,apple,0.933463,2284.740479,474.931152,2553.453613,641.356934
478,SK-C-535.png,knife,0.992804,1343.378784,1899.955200,1685.898560,2035.206787
479,SK-C-535.png,person,0.997663,444.870605,498.256317,1440.576050,1634.386597
480,SK-C-535.png,cat,0.975474,1837.809082,2060.239990,2212.036865,2622.857422


In [165]:
q = cur.execute(f"""SELECT painting_id,object_name,confidence,min_x,min_y,max_x,max_y FROM Objects WHERE painting_id = '{painting}' AND source = 'GOOGLE_VISION' OR source = 'HUGGING_FACE' OR source ='GVHF'""")
cols = [column[0] for column in q.description]
objects = pd.DataFrame.from_records(data = q.fetchall(), columns = cols)
objects


,source,painting_id,object_name,confidence,min_x,min_y,max_x,max_y
0,HUGGING_FACE,RP-F-2004-71.png,bowl,0.988330,1583.452881,1669.087402,2487.787354,2012.018066
1,HUGGING_FACE,RP-F-2004-71.png,vase,0.986889,784.371338,984.209656,983.907532,1825.298096
2,HUGGING_FACE,RP-F-2004-71.png,bowl,0.910185,57.069508,1744.722046,428.505676,1987.722900
3,HUGGING_FACE,RP-F-2004-71.png,vase,0.999239,1103.981567,1082.783203,1687.672363,1800.974121
4,HUGGING_FACE,RP-F-2014-7-1-3.png,vase,0.997660,633.093994,482.287903,1108.414062,1163.364258
...,...,...,...,...,...,...,...,...
485,GOOGLE_VISION,SK-A-3123.png,person,0.735400,971.949995,180.254438,2468.277663,1811.954493
486,GOOGLE_VISION,SK-A-3123.png,person,0.734460,1014.958024,227.704938,2307.630777,1742.877000
487,GOOGLE_VISION,SK-A-3123.png,book,0.644178,141.700944,462.354613,1165.431961,1325.652407
488,GOOGLE_VISION,SK-A-3123.png,luggage & bags,0.619303,70.892810,1272.428529,894.862339,1766.952910
